Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
# Feature engineering: cluster the latitude and longitude coordinates
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20)
train['cluster'] = kmeans.fit_predict(train[['longitude', 'latitude']])
test['cluster'] = kmeans.predict(test[['longitude', 'latitude']])

In [0]:
# feature engineering: number of other pumps within a certain distance

# setting a .1 default for distance because these are long distances
#   Length of 1° of latitude = always 111.32 km
#   Length of 1° of longitude = 40075 km * cos( latitude ) / 360
def count_neighbors(data, row, max_distance=.1):
  """ 
  returns the number of neighbors by checking distance using 
  lat/lon coordinates, closer than max_distance

  since I'm not calculating euclidian distance (which took far longer), 
  this is essentially a square with radius max_distance
  """
  squared_dist = max_distance**2
  this_point = [data.iloc[row]['longitude'], data.iloc[row]['latitude']]
  count = 0
  
  return len(data[((data['longitude']-this_point[0])**2 < squared_dist) &
                  ((data['latitude']-this_point[1])**2 < squared_dist)])

In [0]:
# Data wrangling I already had from the lectures and other assignments
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
            
    # drop some duplicate and constant columns
    cols_to_drop = ['quantity_group', 'payment', 'recorded_by']
    X = X.drop(columns=cols_to_drop)

    # 'amount_tsh' is 70% 0's, replace those with the mean of all 
    # non-zero values, minus the top 1% (found during exploration)
    X['amount_tsh'] = X['amount_tsh'].replace(0, 641.5)

    # Region code and district code should be categorical (cardinalies 27 & 20)
    X['region_code'] = X['region_code'].astype(str)
    X['district_code'] = X['district_code'].astype(str)

    # We want to treat the new cluster column as categorical
    X['cluster'] = X['cluster'].astype(str)    

    # New feature: number of years from construction_year to date_recorded
    # missing constructino years will result in nans that are imputed later
    X['date_recorded'] = pd.to_datetime(X['date_recorded'])
    X['pump_age'] = X['date_recorded'].dt.year - X['construction_year']

    # Date itself is unnecessary here
    X = X.drop(columns='date_recorded')

    # Use my neighbors function to engineer a new column
    X['neighbor_count'] = X['longitude'] * 0
    for i in range(len(X)):
      if ~np.isnan(X.at[i, 'neighbor_count']):
        X.at[i, 'neighbor_count'] = count_neighbors(X, i, max_distance=.1)
    
    # return the wrangled dataframe
    return X


train = wrangle(train)
test = wrangle(test)

### Use scikit-learn for hyperparameter optimization with RandomizedSearchCV

In [0]:
# Imports
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [0]:
# Set up X and y variables
target = 'status_group'
features = train.columns.drop([target])
X_train = train[features]
y_train = train[target]

# Create the pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(max_depth=50, n_estimators=200, n_jobs=-1)
)

# Set up parameters
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestclassifier__n_estimators': range(50, 501, 50),
    'randomforestclassifier__max_depth': [10, 25, 50, 100, None],
    'randomforestclassifier__max_features': [.10,.25,.5,.75,'auto',None]
}

# Set up randomized search
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=5, 
    cv=5, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

# Run search on training data
search.fit(X_train, y_train);

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 12.6min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 18.4min


In [0]:
# View the best model found
print('Best hyperparameters:', search.best_params_)
print('Cross-validation score:', search.best_score_)

In [0]:
# View all scores
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

In [0]:
# Run the best estimator found on test, then save the resulting submission file
pipeline = search.best_estimator_

X_test = test[features]
y_test = pipeline.predict(X_test)
submission = test[['id']].copy()
submission[target] = y_test
submission.to_csv('jduke_water_pump_cv_submission.csv', index=False)